In [7]:
import psycopg2
import pandas as pd

CREATE CONNECTION TO DATABASE

In [5]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 port=5433 dbname=new_db user=frank password=testing321")
    print("Connected Successfully")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

Connected Successfully


CREATE CURSOR TO EXECUTE QUERIES

In [17]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("NO Cursor created")
    print(e)

RETRIEVE CLEAN DATA TO BE LOADED INTO THE DATABASE

In [23]:
df =pd.read_csv('../data/clean_data.csv')
df.head()

,Unnamed: 0,track_id,type,traveled_d,avg_speed,lat,lon,speed,lon_acc,lat_acc,time
0,0,1,Car,48.85,9.770344,37.977391,23.737688,4.9178,0.0518,-0.0299,0.0
1,1,2,Motorcycle,98.09,19.839417,37.977642,23.737400,16.9759,-0.0361,-0.0228,0.0
2,2,3,Motorcycle,63.80,18.228752,37.977997,23.737264,20.1906,-0.0795,-0.3395,0.0
3,3,4,Motorcycle,145.72,26.229014,37.978135,23.737072,2.7555,-0.0302,0.0948,0.0
4,4,5,Motorcycle,138.01,24.841425,37.978134,23.737103,0.0000,0.0000,0.0000,0.0


In [20]:
df.columns

Index(['Unnamed: 0', 'track_id', 'type', 'traveled_d', 'avg_speed', 'lat',
       'lon', 'speed', 'lon_acc', 'lat_acc', 'time'],
      dtype='object')

In [27]:
clean_df = df.drop(['Unnamed: 0'], axis=1)
clean_df.head()

,track_id,type,traveled_d,avg_speed,lat,lon,speed,lon_acc,lat_acc,time
0,1,Car,48.85,9.770344,37.977391,23.737688,4.9178,0.0518,-0.0299,0.0
1,2,Motorcycle,98.09,19.839417,37.977642,23.737400,16.9759,-0.0361,-0.0228,0.0
2,3,Motorcycle,63.80,18.228752,37.977997,23.737264,20.1906,-0.0795,-0.3395,0.0
3,4,Motorcycle,145.72,26.229014,37.978135,23.737072,2.7555,-0.0302,0.0948,0.0
4,5,Motorcycle,138.01,24.841425,37.978134,23.737103,0.0000,0.0000,0.0000,0.0


In [28]:
first_five = clean_df.head()
first_five

,track_id,type,traveled_d,avg_speed,lat,lon,speed,lon_acc,lat_acc,time
0,1,Car,48.85,9.770344,37.977391,23.737688,4.9178,0.0518,-0.0299,0.0
1,2,Motorcycle,98.09,19.839417,37.977642,23.737400,16.9759,-0.0361,-0.0228,0.0
2,3,Motorcycle,63.80,18.228752,37.977997,23.737264,20.1906,-0.0795,-0.3395,0.0
3,4,Motorcycle,145.72,26.229014,37.978135,23.737072,2.7555,-0.0302,0.0948,0.0
4,5,Motorcycle,138.01,24.841425,37.978134,23.737103,0.0000,0.0000,0.0000,0.0


QUERIES AND EXECUTION

In [13]:
delete_existing = "DROP TABLE IF EXISTS traffic;"


create_new = ''' 
    CREATE TABLE IF NOT EXISTS traffic(
        track_id numeric, 
        type varchar (100), 
        traveled_d float,
        avg_speed float, 
        lat float, 
        lon float,
        speed float, 
        lon_acc float, 
        lat_acc float, 
        time float
   );
    '''

insert_data = ''' 
    INSERT INTO traffic(
        track_id, 
        type, 
        traveled_d,
        avg_speed, 
        lat, 
        lon,
        speed, 
        lon_acc, 
        lat_acc, 
        time)

    VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

CREATE TABLE

In [14]:
cur.execute(delete_existing)
cur.execute(create_new)

In [16]:
conn.commit() #Ensure action is saved permanently into the DB

INSERT DATA

In [29]:
try: 
    for i, row in first_five.iterrows():
        cur.execute(insert_data, list(row))
    
except psycopg2.Error as e:
    print("Could Not Insert Data")
    print(e)


In [31]:
conn.commit()